In [ ]:
import psi4
import numpy as np
from helper_CQED_RHF import *

In [ ]:
psi4.set_memory("2 GB")
psi4.core.set_output_file("output.dat", False)

numpy_memory = 2


# options for H2O
h2o_options_dict = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}

# test_string
# molecule string for H2O
test_string = """
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
symmetry c1
"""

# molecule string for H2O
h2o_string = """
    O      0.000000000000   0.000000000000   0.000000000000
    H      0.000000000000   0.757000000000   0.587000000000
    H      0.000000000000  -0.757000000000   0.587000000000
symmetry c1
"""

In [ ]:
def modify_geometry_string(geometry_string, displacement_array):
    """
    Extracts Cartesian coordinates from a Psi4 geometry string, applies a
    transformation function to the coordinates, and returns a new geometry string.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        transformation_function (callable): A function that takes a NumPy
            array of Cartesian coordinates (N x 3) as input and returns a
            NumPy array of the same shape with the transformed coordinates.

    Returns:
        str: A new Psi4 molecular geometry string with the transformed coordinates.
    """
    lines = geometry_string.strip().split('\n')
    atom_data = []
    symmetry = None

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if line.lower().startswith("symmetry"):
            symmetry = line
            continue
        parts = line.split()
        if len(parts) == 4:
            atom = parts[0]
            try:
                x, y, z = map(float, parts[1:])
                atom_data.append([atom, x, y, z])
            except ValueError:
                # Handle cases where the line might not be atom coordinates
                pass

    if not atom_data:
        return ""

    coordinates = np.array([[data[1], data[2], data[3]] for data in atom_data])

    # Apply the transformation function
    transformed_coordinates = displacement_array  + coordinates

    new_geometry_lines = []
    for i, data in enumerate(atom_data):
        atom = data[0]
        new_geometry_lines.append(f"{atom} {transformed_coordinates[i, 0]:.8f} {transformed_coordinates[i, 1]:.8f} {transformed_coordinates[i, 2]:.8f}")

    new_geometry_string = "\n".join(new_geometry_lines)
    if symmetry:
        new_geometry_string += f"\n{symmetry}"

    return f"""{new_geometry_string}"""

In [ ]:
displacement = np.zeros((3, 3))


test_str = modify_geometry_string(test_string, displacement)
mol = psi4.geometry(test_str)
print(test_str)

num_atoms = len(test_str.strip().split('\n')) - 1
print(num_atoms)

In [ ]:
def run_qedhf_calculation(geometry_string, displacement_array, lambda_list = [0, 0, 0], basis_set='sto-3g'):
    """
    Runs a Psi4 calculation with the given geometry string, basis set, and method.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_array (np.ndarray): An array of displacements to apply to the coordinates.
        lambda_list (list): A list of lambda values for the calculation, defaults to [0, 0, 0].
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.
        

    Returns:
        dict: A dictionary containing the results of the Psi4 calculation.
    """
    # Modify the geometry string with the displacement array
    test_str = modify_geometry_string(geometry_string, displacement_array)
    print("Just entered run_qedhf_calculation")
    print("Modified geometry string:")
    print(test_str)

    # save options to ditionary
    options_dict = {
        'basis' : basis_set,
        'scf_type' : 'pk',
        'e_convergence' : 1e-12,
        'd_convergence' : 1e-12,
        'save_jk' : True,
    }

    # Set up Psi4 options
    psi4.set_options(options_dict)

    lambda_vector = np.array(lambda_list)

    # Run the Psi4 calculation
    pqed_dict = cqed_rhf(lambda_vector, test_str, options_dict)
    energy = pqed_dict['CQED-RHF ENERGY']

    return energy

def compute_cqed_numerical_gradient(geometry_string, lambda_list=[0, 0, 0], basis_set='sto-3g', displacement_unit=0.001):
    """
    Computes the numerical gradient of the CQED-RHF energy with respect to the geometry.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        displacement_array (np.ndarray): An array of displacements to apply to the coordinates.
        lambda_list (list): A list of lambda values for the calculation, defaults to [0, 0, 0].
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        displacement_unit (float): The step size for finite difference calculations, defaults to 0.001

    Returns:
        np.ndarray: The numerical gradient of the CQED-RHF energy with respect to the geometry.
    """
    # Get the number of atoms from the geometry string
    num_atoms = len(geometry_string.strip().split('\n')) - 1
    print("Number of atoms:", num_atoms)
    # Initialize the numerical gradient array
    numerical_gradient = np.zeros((num_atoms, 3))

    displacement_array = np.zeros((num_atoms, 3))
    print("Displacement array:\n")
    print(displacement_array)
    print("modify_geometry_string\n")
    print(geometry_string)

    # Compute the energy at the original geometry
    original_energy = run_qedhf_calculation(geometry_string, displacement_array, lambda_list, basis_set)

    # Compute the energy at each perturbed geometry
    for i in range(num_atoms):

        for j in range(3):

            displacement_array = np.zeros((num_atoms, 3))
            displacement_array[i, j] = displacement_unit
            
            # print the displacement unit vector
            print(displacement_array)

            # Insert code to compute psi4 energy at forward displacement
            e_f = run_qedhf_calculation(geometry_string, displacement_array, lambda_list, basis_set)

            # Insert code to compute psi4 energy at backwards displacement
            displacement_array *= -1
            e_b = run_qedhf_calculation(geometry_string, displacement_array, lambda_list, basis_set)

            # Insert code to compute finite difference along this displacement
            grad_element = (e_f - e_b) / (2 * 1.88973 * displacement_unit)

            # Insert code to store this to the appropriate gradient element
            numerical_gradient[i, j] = grad_element

    return original_energy, numerical_gradient



In [ ]:
# energy for H2O from hilbert package described in [DePrince:2021:094112]
expected_h2o_e = -76.016355284146

# run qedhf calculation for H2O
h2o_energy = run_qedhf_calculation(test_string, displacement,lambda_list = [0, 0, 0.05], basis_set='cc-pVDZ')

#h2o_energy, h2o_gradient = compute_cqed_numerical_gradient(test_string, lambda_list=[0, 0, 0.05], basis_set='cc-pVDZ', displacement_unit=0.001)

# print energy
print(f"H2O energy: {h2o_energy}")
# print expected energy
print(f"Expected H2O energy: {expected_h2o_e}")
# check if energy is close to expected energy
if np.isclose(h2o_energy, expected_h2o_e, rtol=1e-6):
    print("H2O energy is close to expected energy")
else:
    print("H2O energy is not close to expected energy")
    print(f"Difference: {h2o_energy - expected_h2o_e}")

In [ ]:
h2o_energy, h2o_gradient = compute_cqed_numerical_gradient(test_string, lambda_list=[0, 0, 0.05], basis_set='cc-pVDZ', displacement_unit=0.001)
# print gradient
print(f"H2O gradient\n: {h2o_gradient}")

print(np.isclose(h2o_energy, expected_h2o_e, rtol=1e-6), "H2O energy is close to expected energy")

In [ ]:
def compute_psi4_analytical_gradient(geometry_string, basis_set='sto-3g', method='scf'):
    """
    Computes the energy gradient for a given geometry string using Psi4.

    Args:
        geometry_string (str): A Psi4 molecular geometry string.
        basis_set (str): The basis set to use for the calculation, defaults to 'sto-3g'.
        method (str): The quantum chemistry method to use for the calculation, defaults to 'scf'.

    Returns:
        np.ndarray: The energy gradient as a NumPy array.
    """
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # Run the Psi4 calculation
    try:
        psi4.geometry(geometry_string)
        gradient = psi4.gradient(method)
    except Exception as e:
        print(f"Error during Psi4 calculation: {e}")
        return None

    return np.asarray(gradient)




 


In [ ]:
starting_string = """
O 0.0 0.0 0.0
H 0.0 0.757 0.587
H 0.0 -0.757 0.587
symmetry c1
"""

starting_displacement = np.array([[0, 0.0, 0.0], [0, 0, 0,], [0, 0, .0]])

# Run the Psi4 calculation
energy = run_psi4_calculation(starting_string, starting_displacement, basis_set='sto-3g', method='scf')
gradient = compute_psi4_analytical_gradient(starting_string, basis_set='sto-3g', method='scf')

# test the numerical gradient
numerical_gradient = compute_psi4_numerical_gradient(starting_string, displacement_unit=0.01, basis_set='sto-3g', method='scf')

In [ ]:
print("Energy:", energy)
print("Gradient:\n", gradient)
print("Numerical Gradient:\n", numerical_gradient)

# compute error between numerical and analytical gradient
error = numerical_gradient - gradient
print("Error:\n", error)
print("Norm of error:\n",np.linalg.norm(error))

# Function to compute nuclear gradient
$$\frac{\partial E_{nuc}}{\partial x_i}$$ 


In [ ]:
def compute_nuclear_repulsion_gradient(geometry_string):
    """
    Method to compute the nuclear repulsion gradient

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    The nuclear repulsion gradient only depends on the atom identities and positions
    """
    # Define your molecular geometry
    molecule = psi4.geometry(geometry_string)

    # get the nuclear repulsion gradient
    nuclear_repulsion_gradient = np.asarray(molecule.nuclear_repulsion_energy_deriv1())
    
    return nuclear_repulsion_gradient

# Compute the nuclear repulsion gradient using the starting_string
nuclear_repulsion_gradient = compute_nuclear_repulsion_gradient(starting_string)
print("Nuclear Repulsion Gradient:\n", nuclear_repulsion_gradient)


# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(nuclear_repulsion_gradient, _expected_nuclear_gradient):
    print("Nuclear repulsion gradient is correct.")
else:
    print("Nuclear repulsion gradient is incorrect.")
    print("Computed:\n", nuclear_repulsion_gradient)
    print("Expected:\n", _expected_nuclear_gradient)

# Function to compute Fock matrix expressed in the MO basis
$$ F_{ij} = V_{ij} + T_{ij} + 2 \sum_{k}^{occ} (ii|kk) - \sum_{k}^{occ} (ik|kj) $$


In [ ]:
def compute_fock_matrix_term(geometry_string, basis_set='sto-3g', method='scf'):
    """
    Method to compute the Fock matrix

    Arguments
    ---------

    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The Fock matrix is the matrix representation of the Fock operator, which is used in Hartree-Fock calculations.
    To compute the Fock matrix in the MO basis, we need the one-electron and two-electron integrals and the density matrix.
    We will get these from a converged Hartree-Fock calculation.
    """

    # set up the molecule
    molecule = psi4.geometry(geometry_string)

    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # set up the geometry
    psi4.geometry(geometry_string)

    # run the Hartree-Fock calculation
    rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get number of atoms
    n_atoms = molecule.natom()

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array


    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # get the one-electron integrals
    H_ao = np.asarray(mints.ao_kinetic()) + np.asarray(mints.ao_potential())

    # transform H_ao to the MO basis
    H_mo = np.einsum('uj, vi, uv', Cnp, Cnp, H_ao)

    # get the two-electron integrals, use psi4 to transform into the MO basis because that is more efficient
    ERI =  np.asarray(mints.mo_eri(C, C, C, C))

    # Build the Fock matrix
    F = H_mo + 2 * np.einsum("ijkk->ij", ERI[:, :, :n_docc, :n_docc]) 
    F -= np.einsum("ikkj->ij", ERI[:, :n_docc, :n_docc, :] )

    # now compute the overlap gradient and contract with Fock matrix
    overlap_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    overlap_gradient = np.zeros(3 * n_atoms)
    for atom_index in range(n_atoms):

        # Derivatives with respect to x, y, and z of the current atom
        for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index
            # Get overlap derivatives for this atom and Cartesian component
            overlap_derivs[deriv_index, :, :] = np.asarray(mints.mo_oei_deriv1("OVERLAP", atom_index, C, C)[cart_index])
            overlap_gradient[deriv_index] = -2.0 * np.einsum('ii,ii->', F[:n_docc, :n_docc], overlap_derivs[deriv_index, :n_docc, :n_docc])

    return overlap_gradient

In [ ]:
def compute_one_electron_integral_gradient_terms(geometry_string, basis_set='sto-3g', method='scf'):
    """
    NEEDS COMPLETING: Method to compute the one-electron integral gradient terms

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The one-electron integral gradient terms are the derivatives of the one-electron integrals with respect to the nuclear coordinates.
    To compute the one-electron integral gradient terms, we need the one-electron integrals and the nuclear repulsion gradient.
    We will get these from a converged Hartree-Fock calculation.
    """
    # set up the molecule
    molecule = psi4.geometry(geometry_string)
    
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })

    # set up the geometry
    psi4.geometry(geometry_string)

    # run the Hartree-Fock calculation
    rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the number of atoms
    n_atoms = molecule.natom()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array

    # get the Density matrix by summing over the occupied orbital transformation matrix
    Cocc = Cnp[:, :n_docc]
    D = np.einsum("pi,qi->pq", Cocc, Cocc) * 2 # [Szabo:1996] Eqn. 3.145, pp. 139

    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # initialize the one-electron integrals derivative matrices
    kinetic_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    potential_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    
    # just try single dipole deriv
    dipole_derivs = np.asarray(mints.ao_oei_deriv1("DIPOLE", 0)[0])

    # just try single quadrupole deriv
    quadrupole_derivs = np.asarray(mints.ao_oei_deriv1("QUADRUPOLE", 0)[0])

    print("Shape of dipole derivs:", dipole_derivs.shape)
    print("Shape of quadrupole derivs:", quadrupole_derivs.shape)

    kinetic_gradient = np.zeros(3 * n_atoms)
    potential_gradient = np.zeros(3 * n_atoms)


    # loop over all of the atoms
    for atom_index in range(n_atoms):
        # Derivatives with respect to x, y, and z of the current atom
         for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index

            # get the one-electron integral derivatives
            kinetic_derivs[deriv_index] = np.asarray(mints.ao_oei_deriv1("KINETIC", atom_index)[cart_index])
            potential_derivs[deriv_index] = np.asarray(mints.ao_oei_deriv1("POTENTIAL", atom_index)[cart_index])

            # add code to contract kinetic_derivs with D
            kinetic_gradient[deriv_index] = np.einsum("uv,uv->", D, kinetic_derivs[deriv_index, :, :])

            # add code to contract potential_derivs with D
            potential_gradient[deriv_index] = np.einsum("uv,uv->", D, potential_derivs[deriv_index, :, :])

    
    # add code to return the kinet_gradient and potential_gradient
    return kinetic_gradient, potential_gradient

In [ ]:
# compute the overlap gradient ter
overlap_gradient = compute_fock_matrix_term(starting_string)

# compute the one-electron integral gradient terms
kinetic_gradient, potential_gradient = compute_one_electron_integral_gradient_terms(starting_string)



In [ ]:
# check overlap gradient against expected
print("Expected overlap gradient:\n", _overlap_gradient)
print("Computed overlap gradient:\n", overlap_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(overlap_gradient.reshape(3,3), _overlap_gradient):
    print("Overlap gradient is correct.")
else:
    print("Overlap gradient is incorrect.")

# check kinetic gradient against expected
print("Expected kinetic gradient:\n", _kinetic_gradient)
print("Computed kinetic gradient:\n", kinetic_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(kinetic_gradient.reshape(3,3), _kinetic_gradient):
    print("Kinetic gradient is correct.")
else:
    print("Kinetic gradient is incorrect.")

# check potential gradient against expected
print("Expected potential gradient:\n", _potential_gradient)
print("Computed potential gradient:\n", potential_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(potential_gradient.reshape(3,3), _potential_gradient):
    print("Potential gradient is correct.")
else:
    print("Potential gradient is incorrect.")


In [ ]:
def compute_two_electron_integral_gradient_terms(geometry_string, basis_set='sto-3g', method='scf'):
    """
    NEEDS COMPLETING: Method to compute the two-electron integral gradient terms

    Arguments
    ---------
    geometry_string : str
        psi4 molecule string

    basis_set : str
        basis set to use for the calculation, defaults to 'sto-3g'

    method : str
        quantum chemistry method to use for the calculation, defaults to 'scf'

    The two-electron integral gradient terms are the derivatives of the two-electron integrals with respect to the nuclear coordinates.
    To compute the two-electron integral gradient terms, we need the two-electron integrals and the nuclear repulsion gradient.
    We will get these from a converged Hartree-Fock calculation.
    """
    # set up the molecule
    molecule = psi4.geometry(geometry_string)
    
    # Set up Psi4 options
    psi4.set_options({
        'basis': basis_set,
        'scf_type': 'pk',
        'e_convergence': 1e-8,
        'd_convergence': 1e-8,
    })
    
    # set up the geometry
    psi4.geometry(geometry_string)

    # run the Hartree-Fock calculation
    rhf_e, wfn = psi4.energy(method, return_wfn=True)

    # get the number of orbitals and the number of doubly occupied orbitals
    n_orbitals = wfn.nmo()
    n_docc = wfn.nalpha()

    # get the number of atoms
    n_atoms = molecule.natom()

    # get the orbital transformation matrix
    C = wfn.Ca() # -> as psi4 matrix object
    Cnp = np.asarray(C) # -> as numpy array

    # get the Density matrix by summing over the occupied orbital transformation matrix
    Cocc = Cnp[:, :n_docc]
    D = np.einsum("pi,qi->pq", Cocc, Cocc)  # [Szabo:1996] Eqn. 3.145, pp. 139

    # instantiate the MintsHelper object
    mints = psi4.core.MintsHelper(wfn.basisset())

    # initialize the two-electron integrals derivative matrices
    eri_derivs = np.zeros((3 * n_atoms, n_orbitals, n_orbitals, n_orbitals, n_orbitals))
    J_deriv = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    K_deriv = np.zeros((3 * n_atoms, n_orbitals, n_orbitals))
    J_gradient = np.zeros(3 * n_atoms)
    K_gradient = np.zeros(3 * n_atoms)

    # loop over all of the atoms
    for atom_index in range(n_atoms):
        # Derivatives with respect to x, y, and z of the current atom
        for cart_index in range(3):
            deriv_index = 3 * atom_index + cart_index

            # get the two-electron integral derivatives
            eri_derivs[deriv_index] = np.asarray(mints.ao_tei_deriv1(atom_index)[cart_index])

            # add code to contract eri_derivs with D to get J_deriv. J_uv = 2 * sum_ls (uv|ls) D_ls 
            J_deriv[deriv_index] = 2 * np.einsum("uvls,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

            # add code to contract eri_derivs with D to get K_deriv. K_uv = -1 * sum_ls (ul|vs) D_ls
            K_deriv[deriv_index] = -1 * np.einsum("ulvs,ls->uv", eri_derivs[deriv_index, :, :, :, :], D)

            # add code to contract J_deriv with D to get J_gradient
            J_gradient[deriv_index] = np.einsum("uv,uv->", D, J_deriv[deriv_index, :, :])

            # add code to contract K_deriv with D to get K_gradient
            K_gradient[deriv_index] = np.einsum("uv,uv->", D, K_deriv[deriv_index, :, :])

    # add code to return the J_gradient and K_gradient
    return J_gradient, K_gradient

In [ ]:
J_gradient, K_gradient = compute_two_electron_integral_gradient_terms(starting_string)


In [ ]:
# check J gradient against expected
print("Expected J gradient:\n", _coulomb_gradient)
print("Computed J gradient:\n", J_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(J_gradient.reshape(3,3), _coulomb_gradient):
    print("J gradient is correct.")
else:   
    print("J gradient is incorrect.")
# check K gradient against expected
print("Expected K gradient:\n", _exchange_gradient)
print("Computed K gradient:\n", K_gradient.reshape(3,3))
# Compare the computed nuclear repulsion gradient with the expected value
if np.allclose(K_gradient.reshape(3,3), _exchange_gradient):
    print("K gradient is correct.")
else:
    print("K gradient is incorrect.")


In [ ]:
# compute the total energy gradient
total_energy_gradient = nuclear_repulsion_gradient.flatten() + kinetic_gradient + potential_gradient + J_gradient + K_gradient + overlap_gradient
print("Total energy gradient:\n", total_energy_gradient.reshape(3,3))
# Compare the computed total energy gradient with the expected value psi4
if np.allclose(total_energy_gradient.reshape(3,3), gradient):
    print("Total energy gradient is correct.")
else:  
    print("Total energy gradient is incorrect.")
    print("Computed:\n", total_energy_gradient.reshape(3,3))
    print("Expected:\n", gradient)